In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import csv
import numpy

Using TensorFlow backend.


In [12]:
   def getTrainingData(filename): 
        x_train = numpy.array([[]])
        y_train = numpy.array([])
        x_test = numpy.array([[]])
        y_test = numpy.array([])
        category_mapping = {'Fitness': 2, 'Gender': 3, 'Age': 4, 'Utilitarian': 5, 'Social Status': 6, 'Species': 7}
        attribute_mapping = {'Fit':0, 'Fat':1, 'Male':2, 'Female':3, 'Young': 4, 'Old':5, 'More':6, 'Less':7, 'High':8, 'Low':9, 'Hoomans':10, 'Pets':11} 
        with open(filename) as csv_file:
            csv_reader = csv.DictReader(csv_file, delimiter=',')
            line_count = 0
            for row in csv_reader:
                if line_count > 110000:
                    break;
                if (row['ScenarioType'] != row['ScenarioTypeStrict']) or row['ScenarioType'] == 'Random':
                    continue
                if line_count == 0:
                    # print(f'Column names are {", ".join(row)}')
                    line_count += 1
                else:
                    current = numpy.zeros((1, 8))
                    current[0][0] = row['PedPed']
                    current[0][1] = row['CrossingSignal']
                    current[0][category_mapping[row['ScenarioType']]] = 1
                    if line_count <= 100000:
                        if line_count == 1:
                            x_train = current
                        else:
                            x_train = numpy.append(x_train, current, axis=0)
                        y_train = numpy.append(y_train, [attribute_mapping[row['AttributeLevel']]], axis = 0)
                    else:
                        if line_count == 100001:
                            x_test = current
                        else:
                            x_test = numpy.append(x_test, current, axis=0)
                        y_test = numpy.append(y_test, [attribute_mapping[row['AttributeLevel']]], axis = 0)
                    line_count += 1
        return x_train, y_train, x_test, y_test

In [13]:
x_train, y_train, x_test, y_test = getTrainingData('SharedResponses.csv')
batch_size = 128
num_classes = 12
epochs = 12

# input image dimensions
img_rows, img_cols = 1, 8

In [14]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [15]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 2
x_test /= 2
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (100000, 1, 8, 1)
100000 train samples
10000 test samples


In [16]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [17]:
input_shape

(1, 8, 1)

In [32]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(2, 1),
                 activation='relu',
                 input_shape=input_shape, data_format='channels_first'))
model.add(Conv2D(64, (2, 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [33]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [34]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 100000 samples, validate on 10000 samples
Epoch 1/12
100000/100000 [==============================] - 36s 360us/step - loss: 0.7996 - acc: 0.4972 - val_loss: 0.6975 - val_acc: 0.5164
Epoch 2/12
100000/100000 [==============================] - 36s 359us/step - loss: 0.6998 - acc: 0.5080 - val_loss: 0.6959 - val_acc: 0.5180
Epoch 3/12
100000/100000 [==============================] - 32s 321us/step - loss: 0.6978 - acc: 0.5100 - val_loss: 0.6965 - val_acc: 0.5230
Epoch 4/12
100000/100000 [==============================] - 33s 327us/step - loss: 0.6972 - acc: 0.5097 - val_loss: 0.6961 - val_acc: 0.5081
Epoch 5/12
100000/100000 [==============================] - 31s 307us/step - loss: 0.6967 - acc: 0.5095 - val_loss: 0.6924 - val_acc: 0.5163
Epoch 6/12
100000/100000 [==============================] - 31s 314us/step - loss: 0.6966 - acc: 0.5116 - val_loss: 0.6933 - val_acc: 0.5234
Epoch 7/12
100000/100000 [==============================] - 33s 325us/step - loss: 0.6963 - acc: 0.5126

In [35]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6907536380767823
Test accuracy: 0.5243
